In [1]:
#Install the required packages.
%pip install azure.kusto.data azure.kusto.ingest pandas openai

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 8, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 19.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /nfs4/pyenv-2ad5745c-f5d7-4998-94d8-45411e8fd4c5/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


You need to execute the below function code in your ADX resource's Query editor. This creates a function [series_cosine_similarity_fl](https://learn.microsoft.com/en-us/azure/data-explorer/kusto/functions-library/series-cosine-similarity-fl) to perform vector similarity searches.

```.create-or-alter function with (folder = "Packages\\Series", docstring = "Calculate the Cosine similarity of 2 numerical arrays")
series_cosine_similarity_fl(vec1:dynamic, vec2:dynamic, vec1_size:real=double(null), vec2_size:real=double(null))
{
    let dp = series_dot_product(vec1, vec2);
    let v1l = iff(isnull(vec1_size), sqrt(series_dot_product(vec1, vec1)), vec1_size);
    let v2l = iff(isnull(vec2_size), sqrt(series_dot_product(vec2, vec2)), vec2_size);
    dp/(v1l*v2l)
}```

In [2]:
from azure.kusto.data import KustoConnectionStringBuilder,KustoClient
from azure.kusto.ingest import QueuedIngestClient, IngestionProperties
from azure.kusto.data.data_format import DataFormat
from azure.kusto.data.helpers import dataframe_from_result_table
import openai
import pandas as pd

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 10, Finished, Available)

_Keys for services_

In [3]:
aad_app_id = ""
aad_app_key = ""
tenant_id =""
openai.api_key = ""

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 11, Finished, Available)

In [4]:
KUSTO_DATABASE = "Vector"
TABLE_NAME = "VectorTable" #We will be creating first then querying the same table
cluster = "https://<yourcluster>.francecentral.kusto.windows.net"
ingest_cluster = "https://<yourcluster>.francecentral.kusto.windows.net"


connection_string = KustoConnectionStringBuilder.with_aad_application_key_authentication(
    cluster, aad_app_id, aad_app_key,tenant_id)
KUSTO_CLIENT = KustoClient(connection_string)

#Cliengt for Ingestion - when we need ingest embedding in database
kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(ingest_cluster, aad_app_id, aad_app_key, tenant_id)
ingestionclient = QueuedIngestClient(kcsb)

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 12, Finished, Available)

In [5]:
openai.api_type = "azure"
openai.api_base = "https://<your aoai api>.openai.azure.com/"
openai.api_version = "2022-12-01"

def embed(query):
    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(
            input=query,
            deployment_id="text-embedding-ada-002", #replace with your deployment id
            chunk_size=1
    )["data"][0]["embedding"]
    return embedded_query

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 13, Finished, Available)

In [6]:
def createTableAdx(table_name):
  try:
    CREATE_TABLE_COMMAND = f".create table {table_name} (Content: string, Embeddings: dynamic)"
    RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_TABLE_COMMAND)
    print(f"{RESPONSE}")
    return True
  except:
    return False

def EmbedList(content):
    resultList = []
    for page in content:
      Content = page
      Embeddings = embed(page)
      myList = [Content, Embeddings]
      resultList.append(myList)
    df = pd.DataFrame(resultList, columns=['Content', 'Embeddings'])
    return df

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 14, Finished, Available)

In [7]:
doc_content = ["""Nelson Mandela, born on July 18, 1918, in Mvezo, South Africa, was a prominent anti-apartheid activist and the first black President of South Africa. Mandela dedicated his life to fighting against racial segregation and inequality. He spent 27 years in prison for his activism and emerged as a symbol of resistance and reconciliation. Mandela played a crucial role in dismantling apartheid and fostering a peaceful transition to democracy in South Africa. His leadership and commitment to forgiveness and unity earned him worldwide respect, and he was awarded the Nobel Peace Prize in 1993. Nelson Mandela passed away on December 5, 2013, leaving a legacy as a global icon of justice and equality."""]
#doc_content = ["""Mahatma Gandhi, born on October 2, 1869, in Porbandar, India, was a prominent leader and a key figure in India's struggle for independence from British rule. His philosophy of nonviolence, Satyagraha, became a powerful weapon in the fight against injustice and oppression. Gandhi's teachings emphasized the importance of truth, self-discipline, and compassion towards all beings. He led several nonviolent movements, including the famous Salt March, and inspired millions of people with his principles of peace and harmony. Gandhi was tragically assassinated on January 30, 1948, but his legacy as the "Father of the Nation" continues to inspire generations around the world."""]
#doc_content = ["""Abraham Lincoln, born on February 12, 1809, in Kentucky, United States, was the 16th President of the United States. He is best known for his leadership during the American Civil War and his efforts to abolish slavery. Lincoln's Emancipation Proclamation in 1863 declared slaves in Confederate territories free and paved the way for the eventual end of slavery. He delivered the historic Gettysburg Address in 1863, emphasizing the importance of preserving the Union and honoring those who fought for freedom. Lincoln was assassinated on April 14, 1865, by John Wilkes Booth, leaving a lasting impact on American history as one of its most revered presidents."""]

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 15, Finished, Available)

In [8]:
pd_df = EmbedList(doc_content)
print(pd_df)

ingestion_props = IngestionProperties(
    database=KUSTO_DATABASE,
    table=TABLE_NAME,
    data_format=DataFormat.CSV,
)
if createTableAdx(table_name=TABLE_NAME):
    print(f"{TABLE_NAME} Created.")
    #Once table is created write the content
    ingestionclient.ingest_from_dataframe(pd_df, ingestion_properties=ingestion_props)
else:
    print(f"Failed to create table {TABLE_NAME}.")

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 16, Finished, Available)

                                             Content  \
0  Nelson Mandela, born on July 18, 1918, in Mvez...   

                                          Embeddings  
0  [-0.023506982252001762, -0.05131963640451431, ...  
VectorTable Created.


In [9]:
searchedEmbedding="When was Abraham Lincon born"

KUSTO_QUERY = f"{TABLE_NAME} | extend similarity = series_cosine_similarity_fl(dynamic(\""+str(searchedEmbedding)+"\"),Embeddings,1,1) | top 3 by similarity desc "
print(KUSTO_QUERY)
RESPONSE = KUSTO_CLIENT.execute(KUSTO_DATABASE, KUSTO_QUERY)
df = dataframe_from_result_table(RESPONSE.primary_results[0])
content = "\n".join(df['Content'])
print("\n" + content)

StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 17, Finished, Available)

VectorTable | extend similarity = series_cosine_similarity_fl(dynamic("When was Abraham Lincon born"),Embeddings,1,1) | top 3 by similarity desc 

Abraham Lincoln, born on February 12, 1809, in Kentucky, United States, was the 16th President of the United States. He is best known for his leadership during the American Civil War and his efforts to abolish slavery. Lincoln's Emancipation Proclamation in 1863 declared slaves in Confederate territories free and paved the way for the eventual end of slavery. He delivered the historic Gettysburg Address in 1863, emphasizing the importance of preserving the Union and honoring those who fought for freedom. Lincoln was assassinated on April 14, 1865, by John Wilkes Booth, leaving a lasting impact on American history as one of its most revered presidents.
Abraham Lincoln, born on February 12, 1809, in Kentucky, United States, was the 16th President of the United States. He is best known for his leadership during the American Civil War and his ef

In [10]:
prompt_template = f"""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {content}

    Question: {searchedEmbedding}
    Answer:"""

completion = openai.Completion.create(deployment_id="text-davinci-003",
                                     prompt=prompt_template, stop=".", temperature=0)

print(completion)


StatementMeta(, ac4b6a4a-3d7b-41b2-8951-1792cdb23b73, 18, Finished, Available)

{
  "id": "cmpl-7RHhiNiWNIx7neQoH5K8sCq4atibY",
  "object": "text_completion",
  "created": 1686737266,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": " Abraham Lincoln was born on February 12, 1809",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 447,
    "total_tokens": 457
  }
}
